# EDGE-TO: Flexible Fast Python Topology Optimization For Minimum Compliance
In this notebook we provide a quick demo on how to use EDGE-TO for different scenarios.

A few important notes:
- Most memory and compute efficiencies can only be made possible for structured meshes so for the most part this is what you should be doing.
- Multi-Grid solvers for large problems are only avaible for structured meshes.
- Material models do not yet support automatic differentiation so you will have to define both forward and backwards methods for your solvers.
- Currently only linear elasticity physics is provided but you can supply your own stiffness matrix functions for different cases.

NOTE: The first time you import EDGE-TO it will take few seconds to compile some functions.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from EDGETO import *
import numpy as np
import cupy as cp

import matplotlib.pyplot as plt

import pygmsh

# Material Models
First let us take a look at the material model calsses. We provide two types of material models ```SingleMaterial``` and ```PenalizedMultiMaterial```. Both models apply a penalty exponent and compute the gradients for the solver. In both cases you can also define a penaly schedule function. By default the penalty supplied will be applied during optimization. Here we will gradually ramp up penalty from 1 (no penalty) to 3 in the first 50 steps of optimization. Note that the solver will not stop until the schedule penalty and material model penalty are the same. So even if convergence criteria is met during ramp up the solver will continue (This is good because we may quickly hit convergence criteria with low penalty). Now let us start with a single material case first.

In [ ]:
def penalty_schedule(final_penalty, iteration):
    if iteration > 50:
        return final_penalty
    else:
        return np.round(5*(1-np.cos(iteration/50*np.pi))/2)/5 * (final_penalty-1) + 1
        

# lets plot the schedule
plt.figure(figsize=(10, 5))
plt.plot([penalty_schedule(3, i) for i in range(100)])
plt.xlabel('Iteration')
plt.ylabel('Penalty')
plt.title('Penalty schedule')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

Now let's setup the material model

In [ ]:
material = SingleMaterial(void=1e-9, penalty=3, penalty_schedule=penalty_schedule, volume_fraction=0.1)

# Geometry
There are several types of mesh each with some optimization on the compute side. First is a structured mesh which is the most specialized type of mesh and allows for the best memeory and compute efficiency. Second is an unstructured mesh with the same element types (quads, tris in 2D and hexas and tets in 3D), and finally is amixed type unstructured mesh.

In [ ]:
# Structured Mesh (64x32x32)
mesh = StructuredMesh3D(nx=64, ny=32, nz=32, lx=1, ly=0.5, lz=0.5, dtype=np.float64)

# Create a geometry object
with pygmsh.occ.Geometry()as geom:
    # Load the STEP file
    geom.import_shapes("PacMan.step")
    geom.characteristic_length_max = 15
    geom.characteristic_length_min = 15
    mesh_uniform = geom.generate_mesh()

    mesh_uniform = [mesh_uniform.points[:,0:3], mesh_uniform.cells[2].data.astype(int).tolist()]


mesh_uniform = GeneralMesh(np.array(mesh_uniform[0])/1000, np.array(mesh_uniform[1]), dtype=np.float64)

print(f"Is the mesh uniform? {mesh_uniform.is_uniform}")

A quick not on physics. The mesh object takes as input a function which is meant to compute the stiffness (K), Constitutive/Property (D), Strain-Displacement/Gradient (B) matrices and the element Area/Volume. The signiture of the expected function here is ```python f(x0: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]``` where the function takes element node positions as input and returns the K, D, B, A in this order. By default this will use the built in linear ealsticity stiffness we have implemented you will need to change this if you wish to use a different type of FEA physics.

# Kernels & Filters
In our solver we need to supply a kernel for FEA calculations and a Filter kernel for filter application during optimization. Let's see how this is done:

In [ ]:
# Stiffness Kernels
kernel = StructuredStiffnessKernel(mesh)
kernel_uniform = UniformStiffnessKernel(mesh_uniform)

# Filter Kernels
filter_kernel = StructuredFilter3D(mesh, r_min=1.5) # This r_min is a multiple of the smallest edge size in the mesh
filter_kernel_uniform = GeneralFilter(mesh_uniform, r_min=2.0 * 0.015) # In general cases r_min is mesh space, not scaled by mesh size be cassreful!

# Solvers
We provide a few different solvers which you can use. In this notebook we focus on CPU solvers which are available as the following:

```python
    CHOLMOD(kernel: StiffnessKernel)
    SPLU(kernel: StiffnessKernel)
    SPSOLVE(kernel: StiffnessKernel)
    CG(kernel: StiffnessKernel, maxiter=1000, tol=1e-5)
    BiCGSTAB(kernel: StiffnessKernel, maxiter=1000, tol=1e-5)
    GMRES(kernel: StiffnessKernel, maxiter=1000, tol=1e-5)
    MultiGrid(mesh: Union[StructuredMesh2D,StructuredMesh3D],
              kernel: StiffnessKernel, maxiter=1000, tol=1e-5, n_smooth=3,
              omega=0.5 , n_level = 3, cycle='W', w_level=1, coarse_solver='splu',
              matrix_free=False, low_level_tol = 1e-8, low_level_maxiter=5000)
```

The fastest by far in most cases that can be solved practically using CPU will be CHOLMOD. For larger problems MultiGrid is best but it only works with structured meshes.

In [ ]:
solver = MultiGrid(mesh, kernel, maxiter=50, tol=1e-4, n_level=3, w_level=1)
solver_uniform = CG(kernel_uniform, maxiter=500, tol=1e-4)

# Optimizer Instance
Finally all of these will be given to an optimizer instance which will perform optimization using optimality criteria method. We will also use a Plotter instance for visualization.

In [ ]:
optimizer = TopOpt(mesh, material, kernel, solver, filter_kernel, max_iter=300, fun_tol=1e-5, ch_tol=1e-4)
optimizer_uniform = TopOpt(mesh_uniform, material, kernel_uniform, solver_uniform, filter_kernel_uniform, max_iter=300, fun_tol=1e-5, ch_tol=1e-4)

plotter = Plotter(optimizer)
plotter_uniform = Plotter(optimizer_uniform)

In [ ]:
plotter.display_mesh()

In [ ]:
plotter_uniform.display_mesh()

## Setting Forces and Boundary Conditions
You can supply both positions and node indicies for boundary condtions. For the structured mesh we will setup the cantilever problem and for the other cases we will apply fixed points at the four points.

In [ ]:
# remove any previous BCs (not needed here)
optimizer.reset_BC()
optimizer.reset_F()

# Use nodal method for structured mesh
bc_nodes = np.where(np.isclose(mesh.nodes[:,0],0))[0]
bc_values = np.array([[1,1,1]]) # Broadcastable, If needed you can provide one for each point
optimizer.add_BC_nodal(bc_nodes, bc_values)

force_node = np.where(np.logical_and(np.isclose(mesh.nodes[:,0],1), np.isclose(mesh.nodes[:,1],0.25)))[0]
force = np.array([[0,-1, 0]])/force_node.shape[0] # Broadcastable, If needed you can provide one for each point
optimizer.add_F_nodal(force_node, force)

plotter.display()

In [ ]:
optimizer_uniform.reset_BC()
optimizer_uniform.reset_F()

# find nodes on the boundary of the packman circle
bc_nodes = np.isclose(np.linalg.norm(mesh_uniform.nodes[:,0:2] - np.array([0.5,0.5]), axis=1),0.5)
bc_nodes = np.logical_and(bc_nodes, mesh_uniform.nodes[:,0] < 0.05)
bc_nodes = np.where(bc_nodes)[0]
bc_values = np.array([[1,1,1]]) # Broadcastable, If needed you can provide one for each point
optimizer_uniform.add_BC_nodal(bc_nodes, bc_values)
# apply load at the mouth of the packman
upper_mouth = np.logical_and(np.isclose(np.abs(np.dot(mesh_uniform.nodes,np.array([-1,1,0]))),0), mesh_uniform.nodes[:,0] > 0.5)
force_node = np.where(upper_mouth)[0]
force_nodes = force_node[np.isin(force_node, bc_nodes, invert=True)]
force = np.array([[1,-1, 0]])/force_nodes.shape[0]/4 # Broadcastable, If needed you can provide one for each point
optimizer_uniform.add_F_nodal(force_nodes, force)

lower_mouth = np.logical_and(np.isclose(np.abs(np.dot(mesh_uniform.nodes,np.array([1,1,0]))),1.0), mesh_uniform.nodes[:,0] > 0.5)
force_node = np.where(lower_mouth)[0]
force_nodes = force_node[np.isin(force_node, bc_nodes, invert=True)]
force = np.array([[1,1, 0]])/force_nodes.shape[0]/4 # Broadcastable, If needed you can provide one for each point
optimizer_uniform.add_F_nodal(force_nodes, force)

plotter_uniform.display()

# Optimization

In [ ]:
rho, flag, history = optimizer.optimize(save_comp_history=True)

print(f"Optimization converged: {flag}")

plt.figure(figsize=(10, 5))
plt.plot(history['comp_history'],'--' , label='compliance', color="#002b72")
plt.scatter([49], [history['comp_history'][49]], color="#ff7b00", label = "Penalty Warmup Ends", s=100)
plt.xlabel('Iteration')
plt.ylabel('Compliance')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend()
plt.show()

plotter.display_solution(rho)

In [ ]:
rho_uniform, flag, history_uniform = optimizer_uniform.optimize(save_comp_history=True)

print(f"Optimization converged: {flag}")

plt.figure(figsize=(10, 5))
plt.plot(history_uniform['comp_history'],'--' , label='compliance', color="#002b72")
plt.scatter([49], [history_uniform['comp_history'][49]], color="#ff7b00", label = "Penalty Warmup Ends", s=100)
plt.xlabel('Iteration')
plt.ylabel('Compliance')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend()
plt.show()

plotter_uniform.display_solution(rho_uniform)